In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
from keras.models import Sequential, Model
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.layers import Input
from keras.utils.data_utils import get_file
from keras.regularizers import WeightRegularizer
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from theano.ifelse import ifelse

Using Theano backend.


In [4]:
data = pd.read_csv('../data/dfLongterm.csv')
y = pd.read_csv('../data/dfY.csv')

In [5]:
data.head()

,fea1,fea2,fea3,fea4,fea5,fea6,fea7,fea8,fea9,fea10,...,fea24,fea25,fea26,fea27,fea28,fea29,fea30,fea31,fea32,fea33
0,137.14,-210.30,70.38,-1.80,13.52,7.67,9.21,-4.23,3.85,-2.07,...,0.33,0.34,0.41,0.43,0.20,0.17,0.26,0.54,0.42,0.45
1,96.00,-321.79,105.26,16.30,33.23,11.20,28.95,12.95,16.30,2.92,...,0.55,0.59,0.41,0.38,0.27,0.22,0.09,0.05,0.12,0.25
2,174.55,-216.06,88.31,-8.95,41.89,18.97,7.79,-2.24,5.94,0.00,...,0.25,0.21,0.44,0.21,0.09,0.18,0.31,0.67,0.40,0.53
3,49.23,-185.45,85.01,-8.47,3.75,-17.62,6.10,-14.41,-8.74,-7.15,...,0.24,0.50,0.53,0.20,0.26,0.09,0.14,0.17,0.31,0.33
4,120.00,-209.14,69.00,30.20,31.84,14.17,22.39,5.16,12.23,0.63,...,0.03,0.06,0.12,0.22,0.23,0.18,0.17,0.17,0.43,0.82


In [6]:
scaler = StandardScaler()
scaler.fit(data)
data_scaled = scaler.transform(data)

In [7]:
X = np.reshape(data_scaled, [1802,1,33])

In [8]:
X = np.reshape(X, [1802,1,33])

In [11]:
divisor1 = np.mean(y['valence_mean'])
divisor2 = np.mean(y['arousal_mean'])

In [12]:
valence = y['valence_mean'].as_matrix()/divisor1
arousal = y['arousal_mean'].as_matrix()/divisor2

In [13]:
valence.shape, arousal.shape

((1802,), (1802,))

In [14]:
print('Build model...')
main_input = Input(shape=(1, 33), name='main_input')
# train a 2-layer LSTM with one shared layer
l1 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=True, dropout_W=0.2)(main_input) # the shared layer
b1 = BatchNormalization()(l1)

l2_1 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=False, dropout_W=0.2)(b1) # the layer specialized in valence prediction
b2_1 = BatchNormalization()(l2_1)
l2_2 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=False, dropout_W=0.2)(b1) # the layer specialized in arousal prediction
b2_2 = BatchNormalization()(l2_2)

valence_output = Dense(1, init='glorot_uniform', name='valence')(b2_1)
arousal_output = Dense(1, init='glorot_uniform', name='arousal')(b2_2)

model = Model(input=[main_input], output=[valence_output, arousal_output])

opt = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004, clipvalue=3)

model.compile(loss={'valence':'mae', 'arousal':'mae'}, optimizer=opt)
early_stopping = EarlyStopping(monitor='val_loss', patience=42)
model_checkpoint = ModelCheckpoint('output_files/models/model_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)

model.fit(X, {'valence':valence, 'arousal':arousal}, validation_split=0.2, verbose=2, callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=1, nb_epoch=500)

Build model...
Train on 1441 samples, validate on 361 samples
Epoch 1/500
12s - loss: 0.4675 - valence_loss: 0.2229 - arousal_loss: 0.2446 - val_loss: 10.6295 - val_valence_loss: 5.6812 - val_arousal_loss: 4.9483
Epoch 2/500
12s - loss: 0.4501 - valence_loss: 0.2148 - arousal_loss: 0.2353 - val_loss: 9.8479 - val_valence_loss: 5.0503 - val_arousal_loss: 4.7975
Epoch 3/500
12s - loss: 0.4449 - valence_loss: 0.2127 - arousal_loss: 0.2322 - val_loss: 9.4551 - val_valence_loss: 4.9097 - val_arousal_loss: 4.5454
Epoch 4/500
13s - loss: 0.4483 - valence_loss: 0.2140 - arousal_loss: 0.2343 - val_loss: 8.5745 - val_valence_loss: 4.3384 - val_arousal_loss: 4.2361
Epoch 5/500


KeyboardInterrupt: 